In [58]:
import pandas as pd
import os
import sys

In [59]:
df = pd.read_csv("../data/raw/sample_data_catalog_product.csv", encoding="utf-8")
df.count()

sku                              2052
store_view_code                    12
attribute_set_code               2052
product_type                     2052
categories                       2040
                                 ... 
associated_skus                     1
downloadable_links                  0
downloadable_samples                0
configurable_variations           147
configurable_variation_labels     147
Length: 89, dtype: int64

In [60]:
len(df.columns.tolist())
print("Number of columns: ", len(df.columns.tolist()))
df.columns

Number of columns:  89


Index(['sku', 'store_view_code', 'attribute_set_code', 'product_type',
       'categories', 'product_websites', 'name', 'description',
       'short_description', 'weight', 'product_online', 'tax_class_name',
       'visibility', 'price', 'special_price', 'special_price_from_date',
       'special_price_to_date', 'url_key', 'meta_title', 'meta_keywords',
       'meta_description', 'base_image', 'base_image_label', 'small_image',
       'small_image_label', 'thumbnail_image', 'thumbnail_image_label',
       'swatch_image', 'swatch_image_label', 'created_at', 'updated_at',
       'new_from_date', 'new_to_date', 'display_product_options_in',
       'map_price', 'msrp_price', 'map_enabled', 'gift_message_available',
       'custom_design', 'custom_design_from', 'custom_design_to',
       'custom_layout_update', 'page_layout', 'product_options_container',
       'msrp_display_actual_price_type', 'country_of_manufacture',
       'additional_attributes', 'qty', 'out_of_stock_qty',
       'use

In [61]:
df.head(1)

,sku,store_view_code,attribute_set_code,product_type,categories,product_websites,name,description,short_description,weight,...,bundle_sku_type,bundle_price_view,bundle_weight_type,bundle_values,bundle_shipment_type,associated_skus,downloadable_links,downloadable_samples,configurable_variations,configurable_variation_labels
0,24-MB01,NaN,Bag,simple,"Default Category/Gear,Default Category/Gear/Bags",base,Joust Duffle Bag,<p>The sporty Joust Duffle Bag can't be beat -...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
columns_to_keep = [
    'sku',
    'attribute_set_code',
    'product_type',
    'categories',
    'name',
    'description',
    'price',
    'weight',
    'base_image',
    'qty',
    'is_in_stock',
    'related_skus',
    'crosssell_skus',
    'upsell_skus',
	'created_at',
    'updated_at'
]

df = df[columns_to_keep]
df = df.dropna(axis=1, how="all")

print("Number of columns after dropping: ", len(df.columns.tolist()))
df.head(1)

Number of columns after dropping:  15


,sku,attribute_set_code,product_type,categories,name,description,price,base_image,qty,is_in_stock,related_skus,crosssell_skus,upsell_skus,created_at,updated_at
0,24-MB01,Bag,simple,"Default Category/Gear,Default Category/Gear/Bags",Joust Duffle Bag,<p>The sporty Joust Duffle Bag can't be beat -...,34.0,/m/b/mb01-blue-0.jpg,100.0,1.0,NaN,"24-WG086,24-WG083-blue,24-UG01,24-WG085_Group","24-MB02,24-MB03,24-MB05,24-MB06,24-UB02,24-WB0...",6/26/25,6/26/25


In [63]:
import re

def preprocess_description(description):
    description = str(description)
    html_tag_replacements = {
        "<p>": "",
        "</p>": "",
        "<br>": "\n",
        "<li>": "",
        "</li>": "",
        "<ul>": "",
        "</ul>": "",
        "<ol>": "",
        "</ol>": ""
    }

    # Replace known tags with their mapped replacements
    for tag, replacement in html_tag_replacements.items():
        description = description.replace(tag, replacement)

    # Remove any other remaining HTML tags
    description = re.sub(r"<[^>]+>", "", description)

    # Replace multiple commas or newlines with single
    description = re.sub(r",\s*,+", ",", description)  # Remove duplicate commas
    description = re.sub(r"\n\s*\n+", "\n", description)  # Collapse multiple newlines

    # Strip leading/trailing whitespace and commas
    description = description.strip(" ,\n")

    return description

In [64]:
df['description'] = df['description'].apply(preprocess_description)
df.head(1)

,sku,attribute_set_code,product_type,categories,name,description,price,base_image,qty,is_in_stock,related_skus,crosssell_skus,upsell_skus,created_at,updated_at
0,24-MB01,Bag,simple,"Default Category/Gear,Default Category/Gear/Bags",Joust Duffle Bag,The sporty Joust Duffle Bag can't be beat - no...,34.0,/m/b/mb01-blue-0.jpg,100.0,1.0,NaN,"24-WG086,24-WG083-blue,24-UG01,24-WG085_Group","24-MB02,24-MB03,24-MB05,24-MB06,24-UB02,24-WB0...",6/26/25,6/26/25


In [65]:
def preprocess_categories(categories):
	cleaned_categories = []

	categories = str(categories)
	categories = categories.split(',')

	categories = [category.strip() for category in categories]
	for category in categories:
		category = category.split('/')
		cleaned_categories.append(category[-1])

	return ','.join(cleaned_categories)

In [66]:
df['categories'] = df['categories'].apply(preprocess_categories)
df.head(1)

,sku,attribute_set_code,product_type,categories,name,description,price,base_image,qty,is_in_stock,related_skus,crosssell_skus,upsell_skus,created_at,updated_at
0,24-MB01,Bag,simple,"Gear,Bags",Joust Duffle Bag,The sporty Joust Duffle Bag can't be beat - no...,34.0,/m/b/mb01-blue-0.jpg,100.0,1.0,NaN,"24-WG086,24-WG083-blue,24-UG01,24-WG085_Group","24-MB02,24-MB03,24-MB05,24-MB06,24-UB02,24-WB0...",6/26/25,6/26/25


In [70]:
os.makedirs('../data/processed', exist_ok=True)
df.to_csv('../data/processed/processed_data.csv', index=False)